<a href="https://colab.research.google.com/github/takatakamanbou/AdvML/blob/2024/AdvML2024_ex12notebookA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AdvML ex12notebookA

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/AdvML/AdvML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?AdvML)




板書や口頭で補足する前提なので，この notebook だけでは説明が不完全です．


---
## 教師なし学習，クラスタリング，次元削減
---


---
### 準備


In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

from scipy.stats import multivariate_normal
from sklearn.mixture import GaussianMixture

---
### 教師なし学習とは


**教師あり学習** (supervised learning): 個々の学習データが，「入力」とそれに対する「出力の正解」のペアとして与えられる．

**教師なし学習** (unsupervised learning): 学習データは入力のみで構成され，出力の正解は与えられない．

教師なし学習の目的は，大量のデータが与えられたときに，そのデータのもつ規則性や構造を見つけ出すこと．それらを知り，データから有用な情報を抽出することを目指す．
教師あり学習では出力の正解が学習データとして与えられたが，
教師なし学習では，学習データは入力となるもののみが与えられ，出力の正解は与えられない．
データからどのような規則性や構造を見出したいかによって様々な問題設定や手法があり，出力がどのようなものになるかはその選択による．


教師なし学習の問題の代表例

- **クラスタリング**(clustering)
- **次元削減**(次元圧縮とも，dimensionality reduction)

正規分布や GMM の当てはめによる **確率密度推定** (probability density estimation) も，教師なし学習の問題の一種とみなせる．

---
### クラスタリング

**クラスタリング** (clustering) は，大量のデータをいくつかの塊（**クラスタ** (cluster)）に分ける手続き．
クラスタリングの手法は，**階層型クラスタリング** と **非階層型クラスタリング** に大別できる．

**階層型クラスタリング** (hierarchical clustering): 「クラスタAとクラスタBをあわせたものがクラスタPで，クラスタPとクラスタQをあわせたものがクラスタR」というように，階層的になったクラスタを作るクラスタリング手法．

**非階層型クラスタリング** (non-hierarchical clustering): クラスタ同士に上記のような階層構造をつくらないクラスタリング手法．

それぞれ，実際の手法には様々なものがあり，データや問題の性質に応じて使い分けられる．
この授業では，階層型クラスタリングについては説明しない（学部2年次科目「[多変量解析及び演習](https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA)」参照．[MVA2023では第12回](https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2023#ex12)）．



### K-means アルゴリズム

**$K$-means アルゴリズム**（K-平均法， K-means method）は，データ同士の間のユークリッド距離が計算できるデータを対象としたクラスタリングのアルゴリズム．
データをいくつのクラスタに分けるかを予め決めておいて，各データをどれか一つのクラスタに割り振る．
名前の$K$は予め定めるクラスタの数を表す．

例えば，下図左のような2次元のデータに対して，クラスタ数を $K=3$ として$K$-meansアルゴリズムを適用すると，下図右のような結果が得られる．

<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/kmeans.png">

右の図の各データ点は，3つのクラスタのうちのどれに割り振られたかに応じて3色に塗り分けられている．
図中に描かれた3つの★印は，それぞれのクラスタに所属するデータの重心を表している．
$K$-meansアルゴリズムでは，クラスタごとのデータの重心のことを **セントロイド**(centroid)と呼ぶ．

［**K-means アルゴリズム**］

$N$個の$D$次元ベクトルから成るデータ集合
$$
\{ \mathbf{x}_n \in {\cal R}^{D} | n = 1, 2, \ldots, N\}
$$
を学習データとして，これを$K$個のクラスタ $C_1, C_2, \ldots, C_K$ に分ける $K$-means アルゴリズムは，次のようになる．

(0) クラスタごとのセントロイド $\mathbf{c}_{k} \in {\cal R}^{D}$ ($k=1, 2, \ldots, K$)の初期値を適当に決める．

(1) 学習データ $\mathbf{x}_n$ ($n = 1, 2, \ldots, N$) のそれぞれを，次の手順で $C_1, C_2, \ldots, C_K$ のいずれかに割り振る．
- $\mathbf{x}_n$ に対して，$K$個のセントロイドのうち最も距離の小さいものの番号 $y_n$ を次式のように求める．
$$
\newcommand{\argmin}{\mathop{\rm argmin}\limits}
y_n = \argmin_{k=1,2,\ldots,K}\Vert \mathbf{x}_{n} - \mathbf{c}_{k} \Vert^2
$$
- $\mathbf{x}_n$ をクラスタ $C_{y_n}$ に割り振る

(2) 各クラスタに割り振られた学習データたちの重心を求め，その値でそれぞれのセントロイドを更新する．式で書くと次の通り（注）．
$$
\mathbf{c}_{k} = \frac{1}{|C_k|}\sum_{n:y_n = k} \mathbf{x}_n
$$

(3) 結果が一定の条件を満たしていれば終了，さもなくば (1) へ戻る．


※ 注: この式の和は，$y_n$ が $k$ と等しいような $n$ たちについてとる．すなわち，クラスタ $C_k$ に所属するデータの和を意味する．また，$|C_k|$ は集合 $C_k$ の元の数，つまり，クラスタ $C_k$ に割り振られたデータの個数を表す．



$K$-平均法には次のような性質がある．

性質1: $K$-平均法の学習では，次式で表される量が最小化される．
$$
E = \sum_{k=1}^{K}\sum_{n:y_n = k} \Vert \mathbf{x}_n - \mathbf{c}_{k}\Vert^2
$$
この値は，「学習データのそれぞれが割り振られたクラスタのセントロイドとの距離の二乗」の和である．アルゴリズムの繰り返しごとに，この $E$ の値は単調減少する．

性質2: $K$-meansアルゴリズムの学習結果は，初期値のとり方によって変わる．初期値によっては，$E$ の（最小でない）極小解に到達してそれ以上解が変化しなくなる．そのため，実用の際は，何通りかの初期値で学習を繰り返し，$E$の値が最も小さかった結果を採用する，といった方法がとられる．

---
### 実験: 2次元データに K-means アルゴリズムを適用してみる

#### 準備

データと K-means アルゴリズムの学習手続きの関数の定義．

In [ ]:
# 実験用データの入手
df = pd.read_csv('https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/data4kmeans.csv')
dat1 = df[['x1', 'x2']].to_numpy()
dat2 = df[['y1', 'y2']].to_numpy()

In [ ]:
## セントロイドの初期化
#
def initCentroid(X, centroid, seed=None):
    assert X.shape[1] == centroid.shape[1]
    K = centroid.shape[0]
    # 学習データからランダムに K 個を選択して初期セントロイドとする
    N = X.shape[0]
    idx = np.arange(N, dtype=int)
    if seed is not None:
        np.random.seed(seed)
    np.random.shuffle(idx)
    centroid[:] = X[idx[:K], :]

## データをクラスタに割り振る
#
def assignCluster(X, centroid, label):
    assert X.shape[1] == centroid.shape[1] and X.shape[0] == label.shape[0]
    K = centroid.shape[0]
    N = X.shape[0]
    sqe = 0.0
    for n in range(N):
        # 各セントロイドとの距離の二乗を計算
        d = np.sum((X[n, :] - centroid)**2, axis=1)
        # 距離最小のクラスタへ割り振る
        i = np.argmin(d)
        label[n] = i
        sqe += d[i]

    return sqe/N  # 割り振られたセントロイドとの距離の二乗の平均

## セントロイドを計算し直す
#
def updateCentroid(X, centroid, label):
    assert X.shape[1] == centroid.shape[1] and X.shape[0] == label.shape[0]
    K = centroid.shape[0]
    for ik in range(K):
        # ik 番目のクラスタに割り当てられたデータの平均をそのクラスタの新しいセントロイドとする
        centroid[ik, :] = np.mean(X[label==ik, :], axis=0)

#### データその1

次のコードセルをそのまま実行すると，`dat1` という変数に格納された2次元データの散布図を描く．

In [ ]:
X = dat1  #　データその1
#X = dat2  # データその2

# クラスタリング結果描画用のデータ
xmin, xmax = -5, 5
ymin, ymax = -5, 5
p = np.dstack(np.mgrid[xmin:xmax:0.05, ymin:ymax:0.05])
P = p.reshape((-1, p.shape[2]))

# 散布図
fig, ax = plt.subplots(facecolor="white", figsize=(4, 4))
ax.scatter(X[:, 0], X[:, 1])
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_aspect('equal')
plt.show()

次のコードセルは，K-means アルゴリズムの初期化の手続きである．

In [ ]:
## 初期化

K = 3  # クラスタ数

N, D = X.shape
centroid = np.empty((K, D)) # セントロイド
label = np.empty(N, dtype=int) # 各学習データの所属するセントロイドの番号
labelP = np.empty(P.shape[0], dtype=int)

initCentroid(X, centroid)  # セントロイドを初期化

i = 0  # K-平均法の繰り返し回数

上のセルを実行して初期化を行ったのち，下のセルを実行すると，K-meansアルゴリズムの繰り返しの1回分を実行することができる．
繰り返し実行して，学習が進む様子を観察しよう．

In [ ]:
## 1ステップの学習を実行

# データを各クラスタに割り振る
msqe = assignCluster(X, centroid, label)
assignCluster(P, centroid, labelP)

# 現在のクラスタ割り振り結果を描画
colors = seaborn.color_palette(n_colors=K)
fig = plt.figure(facecolor="white", figsize=(10, 5))
ax0 = fig.add_subplot(121)
ax0.set_xlim(xmin, xmax)
ax0.set_ylim(ymin, ymax)
ax0.set_aspect('equal')
for ik in range(K):
    Xk = X[label==ik, :]
    Pk = P[labelP==ik, :]
    ax0.scatter(Xk[:, 0], Xk[:, 1], color=colors[ik])
    ax0.plot(centroid[ik, 0], centroid[ik, 1], color='white', marker='*', markerfacecolor=colors[ik], markersize=25)
    ax0.scatter(Pk[:, 0], Pk[:, 1], marker='.', alpha=0.1, color=colors[ik])
    ax0.text(xmin+0.5, ymin+0.5, f'step{i}: {msqe:.3f}', size=20)

# セントロイドを更新
updateCentroid(X, centroid, label)
i += 1

# 新しいセントロイドを表示
ax1 = fig.add_subplot(122)
ax1.set_xlim(xmin, xmax)
ax1.set_ylim(ymin, ymax)
ax1.set_aspect('equal')
for ik in range(K):
    Xk = X[label==ik, :]
    ax1.scatter(Xk[:, 0], Xk[:, 1], color=colors[ik])
    ax1.plot(centroid[ik, 0], centroid[ik, 1], color='white', marker='*', markerfacecolor=colors[ik], markersize=25)

plt.show()

K-means アルゴリズムは，パラメータの初期値（初期セントロイドの選び方）によって学習結果が変わる．初期化からやり直してみよう．

さらに，K の値を変えて実験してみよう．

#### データその2


「データその1」の最初のコードセルの最初の2行のコメントを付け替えることで，`dat2` という変数に入ったデータでも実験することができる．

---
### GMM を用いたクラスタリング

混合正規分布モデル(GMM)を用いてクラスタリングすることもできる．
GMM は

$$
\begin{aligned}
p(\mathbf{x}) = \sum_{k=1}^{K} w_k {\cal N}(\mathbf{x}; \mathbf{\mu}_k, \Sigma_k)
\end{aligned}
$$

という式で与えられる．データが $K$ 個の正規分布のうちどれから生成されたかを表す潜在変数を $z$ とおくと，

$$
\begin{aligned}
p(z = k) &= w_k\\
p(\mathbf{x}|z = k) &= {\cal N}(\mathbf{x}; \mathbf{\mu}_k, \Sigma_k)
\end{aligned}
$$

である．よって，

$$
p(\mathbf{x}, z = k) = p(\mathbf{x}|z = k) p(z = k) = p(z = k|\mathbf{x}) p(\mathbf{x})
$$

より，

$$
\begin{aligned}
p(z = k|\mathbf{x}) &= \frac{p(\mathbf{x}|z = k) p(z = k)}{p(\mathbf{x})}\\
&= \frac{w_k {\cal N}(\mathbf{x}; \mathbf{\mu}_k, \Sigma_k)}{\sum_{j=1}^{K} w_j {\cal N}(\mathbf{x}; \mathbf{\mu}_j, \Sigma_j)}
\end{aligned}
$$

が成り立つ．


学習データに GMM を当てはめたのちに，データ $\mathbf{x}$ に対して $p(z = k|\mathbf{x}) $ ($k = 1, 2, \ldots, K$) を求めれば，$k$ 番目の正規分布が表すクラスタにこのデータが所属する確率を求めることができる．

K-means アルゴリズムでは，個々のデータは $K$ 個のクラスタのうちのどれか一つに割り当てられた．GMM を用いる場合は，上記のように確率的にクラスタに割り当てることができる．前者のようなクラスタの割り当てかたを hard-assignment といい，後者の割り当て方を soft-assignment という（注）．

※注: $\underset{k}{\operatorname{argmax}}p(z = k|\mathbf{x})$ として hard-assignment 化することも可能．


次の2つのコードセルを実行すると，↑と同じデータを GMM を用いてクラスタリングすることができる．

In [ ]:
X = dat1  #　データその1
#X = dat2  # データその2

# GMM の当てはめ
K = 3
gmm = GaussianMixture(n_components=3, covariance_type='full')
gmm.fit(X)

# グラフ描画用のグリッドデータの作成
xmin, xmax = -5, 5
ymin, ymax = -5, 5
x_mesh, y_mesh = np.mgrid[xmin:xmax:(xmax-xmin)/100, ymin:ymax:(ymax-ymin)/100]
X_mesh = np.dstack((x_mesh, y_mesh))

# グラフ
fig, ax = plt.subplots()
ax.scatter(X[:, 0], X[:, 1])

# Gaussian を当てはめた結果
for k in range(K):
    mu = gmm.means_[k]
    cov = gmm.covariances_[k]
    ax.contour(x_mesh, y_mesh, multivariate_normal.pdf(X_mesh, mean=mu, cov=cov))
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_aspect('equal')

In [ ]:
# 平面上の各点の p(z=k|x) の値を求める
p = gmm.predict_proba(X_mesh.reshape((-1, 2))).T
pp = p.reshape((K, X_mesh.shape[0], X_mesh.shape[1]))

# 学習データをクラスタへ hard-assign
y = gmm.predict(X)
mu = gmm.means_

# グラフ
fig = plt.figure(figsize=(9, 6))

# 平面を p(z=k|x) の値で塗り分け
ax0 = fig.add_subplot(121)
cmap = ['Blues', 'Oranges', 'Greens']
for k in range(K):
    ax0.scatter(X[y==k, 0], X[y==k, 1])
    ax0.contourf(x_mesh, y_mesh, pp[k], levels=[0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], cmap=cmap[k], alpha=0.3)
    ax0.plot(mu[k, 0], mu[k, 1], color='white', marker='*', markerfacecolor=colors[k], markersize=25)
ax0.set_xlim(xmin, xmax)
ax0.set_ylim(ymin, ymax)
ax0.set_aspect('equal')

# 学習データ点の塗り分け
ax1 = fig.add_subplot(122)
colors = seaborn.color_palette(n_colors=K)
for k in range(K):
    ax1.scatter(X[y == k, 0], X[y == k, 1], label=f'cluster {k}')
    ax1.plot(mu[k, 0], mu[k, 1], color='white', marker='*', markerfacecolor=colors[k], markersize=25)
ax1.set_xlim(xmin, xmax)
ax1.set_ylim(ymin, ymax)
ax1.legend()
ax1.set_aspect('equal')


plt.show()

左図は，平面全体を $p(z=k|\mathbf{x})$ の値によって塗り分けたものである．右図は，学習データ点を hard-assigmnent した結果を示している．

---
### 主成分分析

次元削減の目的は，与えられたデータを，そこに含まれる情報をなるべく損なわないようにしつつ，より低次元へと変換することである．
統計手法である **主成分分析** (Principal Component Analysis, PCA)では，次元削減の変換を線形変換によって構成する手法の一つである．

以下では，$N$個の$D$次元ベクトルから成る学習データ

$$
\{ \mathbf{x}_n \in {\cal R}^{D} | n = 1, 2, \ldots, N\}
$$

が与えられるとして，これを $H(\leq D)$ 次元に次元削減する場合を考える．また，話を簡単にするために，学習データの平均は $\mathbf{0}$ である，すなわち

$$
\frac{1}{N}\sum_{n=1}^{N}\mathbf{x}_{n} = \mathbf{0}
$$

と仮定する（$\mathbf{0}$でない場合の扱いについては後述する）．

#### $1$次元に次元削減する場合

---
［主成分分析の問題設定（1次元にする場合）］

上で説明したような学習データが与えられるとする．
このとき，大きさ $1$ の $D$ 次元ベクトル $\mathbf{u}$ を用いて，次式によって $D$次元ベクトル $\mathbf{x}$ を1つの実数値 $z_n$ に変換することを考える．

$$
z_n = \mathbf{u}\cdot \mathbf{x}_n \quad (n = 1, 2, \ldots, N)
$$

ベクトル $\mathbf{u}$ を，大きさ1($\Vert\mathbf{u}\Vert = 1$)のベクトルの中で $z_n$の分散が最大となるように定めたい．

---

この問題の解，つまり，$z_n$ の分散を最大にする $\mathbf{u}$ は，「**データ$\mathbf{x}_n$の分散共分散行列の固有ベクトルのうち，最大の固有値に対応するもの**」となる（導出過程は後述）．
したがって，学習データから分散共分散行列を計算し，その固有値固有ベクトルを求めれば，主成分分析による次元削減を実現できる．




上記の解の導出過程の概略を以下に示す．

まず，$\mathbf{x}_n$ の平均が $\mathbf{0}$ なので，$z_n$ の平均も $0$ となる（Q1）．そのため，$z_n$ の分散は

$$
\frac{1}{N}\sum_{n=1}^{N}z_n^2
$$

と表せる．このとき，

$$
\frac{1}{N}\sum_{n=1}^{N}z_n^2 = \mathbf{u}^{\top}V\mathbf{u}
$$

と書ける（Q2）．ただし，$V$ は学習データの分散共分散行列である．



ここで求めたいのは，$\{ z_n \}$ の分散 $\mathbf{u}^\top V\mathbf{u}$ を最大にするベクトル $\mathbf{u}$ である．ただし，$\mathbf{u}$ には $\Vert\mathbf{u}\Vert = 1$ という条件がある．このような制約条件付きの最適化問題を解く定番の手法は，「ラグランジュの未定乗数法」である（注）．
この問題の場合，ラグランジュ乗数を $\lambda$ として，

$$
L = \mathbf{u}^{\top}V\mathbf{u} - \lambda(\mathbf{u}^{\top}\mathbf{u} - 1)
$$

とおけば，$\frac{\partial L}{\partial \mathbf{u}} = \mathbf{0}$ を満たす $\mathbf{u}$ が解の候補となる．

<span style="font-size: 75%">
※注: 「ラグランジュの未定乗数法」は，大学初年次の微積分で学んでいる...かも．興味のあるひとは数学の参考書を調べてね．
</span>

ここで，

$$
\frac{\partial L}{\partial \mathbf{u}} = 2V\mathbf{u} - 2\lambda\mathbf{u}
$$

となる（注）ので，解は

$$
V\mathbf{u} = \lambda\mathbf{u}
$$

を満たさねばならない．この式より，解の候補は $V$ の単位固有ベクトルであることがわかる．
この式を $L$ の式に代入すると，

$$
L = \lambda\mathbf{u}^\top\mathbf{u} - \lambda\cdot 0 = \lambda
$$

となるので，この問題の解は，$V$の最大固有値に対する固有ベクトルであることが分かる．



<span style="font-size: 75%">
※注: $\frac{\partial L}{\partial \mathbf{u}}$ は，「$L$を $\mathbf{u}$ の各要素で偏微分したものをならべたベクトル」です．
</span>

**★★★ やってみよう ★★★**

Q1, Q2 を証明しよう．

#### $2$次元以上に次元削減する場合

主成分分析の目的は，線形変換によって，「元のデータの分散をなるべく保つような」次元削減を行うことである（厳密な問題の定式化は省略する）．

$D$ 次元ベクトル（$D\times 1 $ 行列） $\mathbf{x}$ を $H$ 次元ベクトル（$H\times 1$行列） $\mathbf{z}$  に線形変換する式を

$$
\mathbf{z} = W\mathbf{x}
$$

と表すと，この式は

$$
\mathbf{z} = \begin{pmatrix}
z_1\\
z_2\\
\vdots\\
z_H
\end{pmatrix} = \begin{pmatrix}
\mathbf{w}_1\cdot\mathbf{x}\\
\mathbf{w}_2\cdot\mathbf{x}\\
\vdots\\
\mathbf{w}_H\cdot\mathbf{x}
\end{pmatrix}
$$

と書ける．このような線形変換 $W$ を定めることは，$H$個のベクトル $\mathbf{w}_1, \mathbf{w}_2,\ldots,\mathbf{w}_H$ を定めることと同じである．

このとき，「元のデータの分散を保つ」という意味で最適な線形変換 $W$ は，次のようになる．



［主成分分析の問題の解］

$N$個の$D$次元ベクトルから成る学習データが与えられるとする．
$$
\{ \mathbf{x}_n \in {\cal R}^{D} | n = 1, 2, \ldots, N\}
$$
ただし，このデータの平均は $\mathbf{0}$ とする．
また，$\mathbf{x}$ の分散共分散行列 $V = \frac{1}{N}\sum_{n=1}^{N}\mathbf{x}_n\mathbf{x}_n^{\top}$ の固有値を $\lambda_1 \geq \lambda_2 \geq \ldots \geq \lambda_D$とし，これらに対応する単位固有ベクトルをそれぞれ $\mathbf{u}_1, \mathbf{u}_2,\ldots,\mathbf{u}_D$ とおく．

$H\times D$行列 $W$ によって

$$
\mathbf{z} = W\mathbf{x} = \begin{pmatrix}
\mathbf{w}_1\cdot\mathbf{x}\\
\mathbf{w}_2\cdot\mathbf{x}\\
\vdots\\
\mathbf{w}_H\cdot\mathbf{x}
\end{pmatrix}
$$

と次元削減するとき，主成分分析の目的を実現するためには，
ベクトル $\mathbf{w}_h$ ($h = 1, 2, \ldots, H$) を $\mathbf{u}_h$ の向きにとればよい（注）．

ここで，$\lambda_1$ から $\lambda_H$ （$H \leq D$）に対応する固有ベクトル $\mathbf{u}_1, \mathbf{u}_2, \ldots, \mathbf{u}_H$ をならべた，次のような行列 $U_H$ を考える．

$$
U_H = (\mathbf{u}_1\  \mathbf{u}_2\  \ldots\ \mathbf{u}_H )
$$

$U_H$ は $D \times H$ 行列である．このとき，主成分分析の変換行列 $W$ は，

$$
W = U_H^{\top} = \begin{pmatrix} \mathbf{u}_1^{\top} \\ \mathbf{u}_2^{\top} \\ \vdots \\ \mathbf{u}_H^{\top} \end{pmatrix}
$$

となる．

※ 注: ここで考えている固有ベクトルは「単位」固有ベクトル（大きさが1，すなわち $||\mathbf{u}_h||=1$）に限定しているが，符号の不定性がある（$\mathbf{u}_h$も$−\mathbf{u}_h$も固有ベクトル）．したがって，$\mathbf{w}_h = -\mathbf{u}_h$ としてもよい．


<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/pcafig06b.png">

学習データ $\{ \mathbf{x}_n \}$ の平均が $\mathbf{0}$ でない場合の主成分分析を考える．
この場合，データの平均を $\mathbf{\mu}$ として，つまり

$$
\mathbf{\mu} = \frac{1}{N}\sum_{n=1}^{N}\mathbf{x}_n
$$

として，$\mathbf{x}'_n = \mathbf{x}_n - \mathbf{\mu}$ という変数変換を考えると，
$\mathbf{x}'_n$ の平均は $\mathbf{0}$ である．また，その分散共分散行列 $V$ は

$$
V = \frac{1}{N}\sum_{n=1}^{N} \mathbf{x}'_n{\mathbf{x}'_n}^{\top} = \frac{1}{N}\sum_{n=1}^{N} (\mathbf{x}_n - \mathbf{\mu})({\mathbf{x}_n} - \mathbf{\mu})^{\top}
$$

となり，$\mathbf{x}_n$ の分散共分散行列に等しい．したがって，$V$ の固有値と固有ベクトルを求めて，次元削減の変換を次のようにすればよい．

$$
\mathbf{z} = U_H^{\top}(\mathbf{x} - \mathbf{\mu}) = \begin{pmatrix}
\mathbf{u}_1\cdot(\mathbf{x} - \mathbf{\mu})\\
\mathbf{u}_2\cdot(\mathbf{x} - \mathbf{\mu})\\
\vdots\\
\mathbf{u}_H\cdot(\mathbf{x} - \mathbf{\mu})
\end{pmatrix}
$$

